# Statistical and Machine Learning - Assignment 7

#### By - Swasthik Vellingiri Kowsalya

In [2]:
install.packages("LiblineaR")
library(LiblineaR)     # LR Lasso (l1)
library(mlr)
library(nnet)          # class.ind() function
library(neuralnet)     # Deep Neural Networks
#library(LiblineaR)     # LR Lasso (l1)
#library(randomForest)  # Random Forest
#library(adabag)        # Boosting
library(e1071)         # SVM
library(ggplot2)       # Visualization
library(plotly)        # 3D visualization
library(neuralnet)

# Import data
library(ISLR)      # Data from the course book
library(MASS)      # Boston housing dataset
library(datasets)  # US crime dataset

# Resize plot
library(repr)  # String and binary representations

Warning message:
"package 'nnet' was built under R version 3.6.3"Warning message:
"package 'neuralnet' was built under R version 3.6.3"Warning message:
"package 'e1071' was built under R version 3.6.3"
Attaching package: 'e1071'

The following object is masked from 'package:mlr':

    impute


Attaching package: 'plotly'

The following object is masked from 'package:ggplot2':

    last_plot

The following object is masked from 'package:stats':

    filter

The following object is masked from 'package:graphics':

    layout

Warning message:
"package 'MASS' was built under R version 3.6.3"
Attaching package: 'MASS'

The following object is masked from 'package:plotly':

    select



In [4]:
df = read.csv('C:/Users/svellingirikowsalya/Desktop/Statistical and Machine Learning/SML_Section7/SML_Section7/data/bankruptcy_prediction/data.csv')
ncol(df)

[1] 96

## Q1: Randomly divide data into train/test as 80/20 ( set.seed = 1)

In [55]:
set.seed = 1

In [5]:
split1 = sample(c(rep(0, 0.8 * nrow(df)), rep(1, 0.2 * nrow(df))))

In [6]:
data_train <- df[split1 == 0, ]
data_test <- df[split1 == 1, ]

In [7]:
dim(data_test)

[1] 1364   96

In [8]:
dim(data_train)

[1] 5455   96

## Q2: Build a NN model with 1 hidden layer of 30 neurons, sigmoid activation function.

In [25]:
nn_md1 <- neuralnet(Bankrupt.~.,          # The formula
                     data_train,             # Training data
                     hidden=c(30),      # Size of the hidden layers
                     #threshold=0.1,          # Stopping criteria, a.k.a convergence
                     stepmax=5000,            # Maximum training step
                     rep=2,                  # Number of training repeat, a.k.a epoch
                     lifesign='full',         # Print training process
                     lifesign.step=5000,      # Print out every 5000 steps
                     algorithm='rprop+',      # Algorithm to calculate the network, 'rprop+'=resilient backpropagation
                     learningrate=0.01,       # Learning rate, only use for traditional backpropagation
                     err.fct='sse',           # Error function, sse=sum square error, ce=cross-entropy
                     act.fct="logistic",      # Activation function, 'logistic' or 'tanh'
                     linear.output=F
                    )

hidden: 30    thresh: 0.01    rep: 1/2    steps:       8	error: 92.49689	time: 0.16 secs
hidden: 30    thresh: 0.01    rep: 2/2    steps:      14	error: 92.49176	time: 0.3 secs


In [26]:
# Make prediction and evaluation on train

y_train_pred <- predict(nn_md1, data_train[, c(2:96)])
mean((max.col(y_train_pred)-1) == data_train$Bankrupt.)

[1] 0.9660862

In [27]:
# Make prediction and evaluation on test
y_test_pred <- predict(nn_md1, data_test[, c(2:96)])
mean((max.col(y_test_pred)-1) == data_test$Bankrupt.)

[1] 0.9743402

## Q3: Build a deep NN model with multiple hidden layers (of your choice) and sigmoid activation function.

In [58]:
nn_md2 <- neuralnet(Bankrupt.~.,          # The formula
                     data_train,             # Training data
                     hidden=c(10,10,10,10),      # Size of the hidden layers
                     #threshold=0.1,          # Stopping criteria, a.k.a convergence
                     stepmax=10000,            # Maximum training step
                     rep=3,                  # Number of training repeat, a.k.a epoch
                     lifesign='full',         # Print training process
                     lifesign.step=5000,      # Print out every 5000 steps
                     algorithm='rprop+',      # Algorithm to calculate the network, 'rprop+'=resilient backpropagation
                     learningrate=0.01,       # Learning rate, only use for traditional backpropagation
                     err.fct='sse',           # Error function, sse=sum square error, ce=cross-entropy
                     act.fct="logistic",      # Activation function, 'logistic' or 'tanh'
                     linear.output=F
                    )

hidden: 10, 10, 10, 10    thresh: 0.01    rep: 1/3    steps:     294	error: 88.55562	time: 7.16 secs
hidden: 10, 10, 10, 10    thresh: 0.01    rep: 2/3    steps:      64	error: 89.31346	time: 1.5 secs
hidden: 10, 10, 10, 10    thresh: 0.01    rep: 3/3    steps:      29	error: 89.35688	time: 0.79 secs


In [59]:
# Make prediction and evaluation on train

y_train_pred <- predict(nn_md2, data_train[, c(2:96)])
mean((max.col(y_train_pred)-1) == data_train$Bankrupt.)

[1] 0.9660862

In [60]:
# Make prediction and evaluation on test
y_test_pred <- predict(nn_md2, data_test[, c(2:96)])
mean((max.col(y_test_pred)-1) == data_test$Bankrupt.)

[1] 0.9743402

## Q4: Build 5 other classification models and compare with the 2 previous NN models.

In [41]:
# Define the ML classification task

train_task <- mlr::makeClassifTask(id ='MNIST_train', data=data_train, target='Bankrupt.')
test_task <- mlr::makeClassifTask(id='MNIST_test', data=data_test, target='Bankrupt.')

In [61]:
# Logistic Regression Lasso (l1)
learner <- mlr::makeLearner('classif.LiblineaRL1LogReg')  # Register a machine learning model
model <- mlr::train(learner, train_task)
pred_test <- predict(model, task=test_task, proba=T)
lasso = performance(pred_test, measures=acc)

In [64]:
# k-Nearest Neighbor (k=50)
learner <- makeLearner('classif.knn', k=50)
model <- mlr::train(learner, train_task)
pred_test <- predict(model, task=test_task)
knn = performance(pred_test, measures=acc)

In [65]:
# LDA (drop zero-variance features)
learner <- makeLearner('classif.lda')
model <- mlr::train(learner, filterFeatures(train_task, method='variance', threshold=0.1))
#model <- mlr::train(learner, train_task)
pred_test <- predict(model, task=test_task)
lda = performance(pred_test, measures=acc)

In [66]:
# Decision Tree
learner <- mlr::makeLearner('classif.rpart')  # Register a machine learning model
model <- mlr::train(learner, train_task)
pred_test <- predict(model, task=test_task)
DT = performance(pred_test, measures=acc)

In [67]:
# SVM
learner <- makeLearner('classif.svm', scale=FALSE, kernel='linear')  # linear,polynomial,radial,sigmoid
model <- mlr::train(learner, train_task)
pred_test <- predict(model, task=test_task)
svm = performance(pred_test, measures=acc)

In [79]:
# Comaparison

# Both the neural Network models gave the same accuracy and hence only one result is included in the final table
 
Models <- c('NN_train','NN_test','Lasso_reg', 'KNN', 'LDA', 'Decision_trees','SVM')
Accuracy <- c(mean((max.col(y_train_pred)-1) == data_train$Bankrupt.),mean((max.col(y_test_pred)-1) == data_test$Bankrupt.),lasso, knn, lda, DT,svm)

df <- data.frame(Models, Age)
df

Models,Age
NN_train,0.9660862
NN_test,0.9743402
Lasso_reg,0.9743402
KNN,0.9743402
LDA,0.9706745
Decision_trees,0.9714076
SVM,0.5821114
